In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd
from datetime import datetime
import statsmodels.api as sm
import itertools
import warnings
warnings.filterwarnings("ignore")# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
series=pd.read_csv("/kaggle/input/corona-virus-report/covid_19_clean_complete.csv")

In [ ]:
cases = pd.read_csv('/kaggle/input/corona-virus-report/day_wise.csv')

In [ ]:
cases.head()

In [ ]:
cases['New recovered'].plot()

In [ ]:
cases.head()

In [ ]:
import matplotlib.pyplot as plt

Forecasting the Covid-19 Situation around the World (Generalised Model Globally)

In [ ]:
cases.isnull().sum()

In [ ]:
cases.shape

In [ ]:
recovered = cases[['Date','New recovered']]

In [ ]:
recovered['Date'].min(),series['Date'].max()

In [ ]:
recov=recovered.loc[(recovered['Date']>'2020-05-16' )& (recovered['Date'] < '2020-07-15')].copy()

In [ ]:
recov=recov.groupby('Date')['New recovered'].sum().reset_index()

In [ ]:
recov.head()

In [ ]:
rtoll = recov.copy()

In [ ]:
rtoll.head()

In [ ]:
rtoll.set_index('Date',inplace=True)

In [ ]:
rtoll.index = pd.to_datetime(rtoll.index)

In [ ]:
rtoll.head()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] =5,5
rtoll.plot()

Using the dicky fuller method for testing stationarity of the data

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
test_results = adfuller(rtoll['New recovered'])

In [ ]:
#Ho: It is Not Stattionary
#HA : It is Stationary

def adfuller_test (deaths):
    result = adfuller(deaths)
    labels=['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+':'+ str(value))
    if(result[1]<=0.05):
        print("Stationary")
    else:
        print("Not Stationary")

In [ ]:
adfuller_test(rtoll['New recovered'])

As we can see that the p value greater them 0.05 and thus data is Not Stationary... Accept Null Hypothesis

Differencing Used To Make Data Stationary

In [ ]:
plt.figure(figsize=(40,8))
plt.plot(np.log(rtoll['New recovered']))

In [ ]:
rtoll['New recovered Diff'] = rtoll['New recovered']-rtoll['New recovered'].shift(1)

In [ ]:
adfuller_test(rtoll['New recovered Diff'].dropna())

In [ ]:
rtoll.isnull().sum()

In [ ]:
rtoll.drop(['New recovered'],axis=1,inplace=True)

******As you can see that the data has become stationary after differencing**

In [ ]:
from pylab import rcParams 
rcParams['figure.figsize'] =18,8
decomposition = sm.tsa.seasonal_decompose(rtoll.dropna(),model='additive')
fig = decomposition.plot()
plt.show()

# **ARIMA MODEL FOR TIME SERIES FORECASTING OF RECOVERED CASES**

In [ ]:
rtoll.head()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1=fig.add_subplot(211)
fig=sm.graphics.tsa.plot_acf(rtoll['New recovered Diff'].iloc[1:],lags=40,ax=ax1)
ax2=fig.add_subplot(212)
fig=sm.graphics.tsa.plot_pacf(rtoll['New recovered Diff'].iloc[1:],lags=40,ax=ax2)

From the Above Graph we can see that p value is 1 , as it shuts of in that position , q=0 because the exponential characteristics is unseen

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
p = q = d = range(0,2)
pdq = list(itertools.product(p,d,q))

In [ ]:
for param in pdq:
        try:
            model=ARIMA(rtoll['New recovered Diff'].dropna(),order=param)
            results=model.fit()
            print('ARIMA{}- AIC:{}'.format(param,results.aic))
        except:
            continue

In [ ]:
model = ARIMA(rtoll['New recovered Diff'].dropna(),order=(0,0,1))

In [ ]:
results=model.fit()

In [ ]:
results.summary().tables[1]

In [ ]:
rtoll['Forecast']=results.predict(start=pd.to_datetime('2020-07-01'),dynamic=False)

In [ ]:
rtoll['Forecast'].tail(5)

In [ ]:
ax=rtoll['New recovered Diff'].plot(label='Observed')
rtoll['Forecast'].plot(ax=ax,label='Predicted',alpha=.7,figsize=(14,7))
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.show()

In [ ]:
rtoll.index[-1]

In [ ]:
#Getting the forecasted value for the next seven days
forecast = results.forecast(steps=7)[0]

In [ ]:
forecast

In [ ]:
from pandas.tseries.offsets import DateOffset
future_dates = [rtoll.index[-1]+ DateOffset(days=x)for x in range(0,7)]

In [ ]:
future_dates_df = pd.DataFrame(index=future_dates[1:],columns=rtoll.columns)

In [ ]:
future_df = pd.concat([rtoll,future_dates_df])

In [ ]:
future_df.tail(6)

In [ ]:
future_df['Forecast'] = results.predict(start='2020-07-01',end='2020-07-20',dynamic=False)
future_df[['New recovered Diff','Forecast']].plot()